In [1]:
%matplotlib inline

import geopandas as gpd

import pandas as pd
import os
import requests
import folium
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
import matplotlib.font_manager as fm

for font in fm.fontManager.ttflist:
    if font.name in ['AppleGothic', 'Malgun Gothic']:
        plt.rcParams['font.family'] = font.name
        break

plt.rcParams['font.family']

['Malgun Gothic']

In [3]:
#2018년 7월 승차일 기준 1~4일의 버스 카드태깅 정보를 담고있습니다.
#미터 / 분
TripChain = pd.read_csv('../Data/PJT001_TripChain.csv')

#17~18년 기준, 경기도 버스 정류장에 대한 정보를 담고있습니다.
# 성보님과 은솔님이 채워주신 데이터 바로 이용
StationTable = pd.read_csv('../Data/station_null.csv')

#2018년 7월 1~4일의 행정동별 이동 인구수 정보를 담고 있습니다.
AreaPeople = pd.read_csv('../Data/PJT001_sk_emd_od.csv')

#버스노선-정류장 매핑 테이블 정보 입니다.
RouteStationInfo = pd.read_csv('../Data/PJT001_routestationinfo.csv')

#
RouteStationMapping = pd.read_csv('../Data/PJT001_routestationmapping.csv')

In [4]:
TripChain.columns=['암호화카드번호', '트랜잭션ID', '환승횟수', '교통카드발행사ID',
       '총이용객수', '사용자구분', '교통수단CD1', '교통수단CD2',
       '교통수단CD3', '교통수단CD4', '교통수단CD5', '버스노선ID1',
       '버스노선ID2', '버스노선ID3', '버스노선ID4', '버스노선ID5',
       '차량ID1', '차량ID2', '차량ID3', '차량ID4',
       '차량ID5', '총통행거리', '총탑승시간', '총소요시간',
       '승차일시1', '승차일시2', '승차일시3', '승차일시4',
       '승차일시5', '하차일시1', '하차일시2', '하차일시3',
       '하차일시4', '하차일시5', '최초승차일시', '최종하차일시',
       '승차역ID1', '승차역ID2', '승차역ID3', '승차역ID4',
       '승차역ID5', '하차역ID1', '하차역ID2', '하차역ID3',
       '하차역ID4', '하차역ID5', '최초승차역ID', '최종하차역ID',
       '총이용금액', '수집건수', '트립체인완료코드']

In [5]:
TripChain.sample()

,암호화카드번호,트랜잭션ID,환승횟수,교통카드발행사ID,총이용객수,사용자구분,교통수단CD1,교통수단CD2,교통수단CD3,교통수단CD4,...,하차역ID1,하차역ID2,하차역ID3,하차역ID4,하차역ID5,최초승차역ID,최종하차역ID,총이용금액,수집건수,트립체인완료코드
15259,900516548272,17,1,9000923,1,1,500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4170099.0,1250,1,;


In [6]:
StationTable.shape

(39457, 14)

In [7]:
RouteStationInfo.sample()

,seq,pr_station_id,bus_line_no,bus_line_no_seq,station_nm,station_id,mobile_no
20959,145235,240000125,11,32,보통리,237000583,42038.0


In [8]:
RouteStationMapping.sample()

,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명
58,경기시내,경기고속,4100200,234000130,41002093,1303


In [9]:
RouteStationMapping[RouteStationMapping['이비노선ID']==228000018]

# 이비노선ID == RouteStationInfo 의 pr_station_id
# 표준노선ID == TripChain의 버스노선ID 1~5

,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명
709,경기시내,경남여객,4100600,228000018,41006260,10-4


In [10]:
TripChain[TripChain['버스노선ID1']==41006260.0 ]

,암호화카드번호,트랜잭션ID,환승횟수,교통카드발행사ID,총이용객수,사용자구분,교통수단CD1,교통수단CD2,교통수단CD3,교통수단CD4,...,하차역ID1,하차역ID2,하차역ID3,하차역ID4,하차역ID5,최초승차역ID,최종하차역ID,총이용금액,수집건수,트립체인완료코드
351351,900458235861,59,3,9000929,1,1,500.0,500.0,500.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4116703.0,1550,3,;
423008,900399600864,23,3,9000929,1,1,500.0,500.0,500.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4117067.0,2150,3,;
474719,900520797175,56,4,9000921,1,1,500.0,500.0,202.0,500.0,...,NaN,NaN,NaN,4108114.0,NaN,NaN,4108114.0,1850,4,;
503222,900026836857,26,4,3101001,1,1,500.0,500.0,500.0,500.0,...,NaN,NaN,NaN,4111686.0,NaN,NaN,4111686.0,2950,4,;
552852,900388380402,8,4,3101915,1,1,500.0,500.0,500.0,500.0,...,NaN,NaN,NaN,4111850.0,NaN,NaN,4111850.0,1650,4,;
599761,900481809000,81,4,9000922,1,1,500.0,500.0,202.0,500.0,...,NaN,NaN,NaN,4135034.0,NaN,NaN,4135034.0,1550,4,;
803957,100500886129,21,3,3104012,1,1,500.0,500.0,500.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4150162.0,1550,3,;
1008017,900397793341,8,4,3101000,1,1,500.0,500.0,500.0,500.0,...,NaN,NaN,NaN,4117031.0,NaN,NaN,4117031.0,1950,4,;


In [161]:
# RouteStationMapping[RouteStationMapping['화성시']]

# 화성시 데이터만 뽑겠습니다

In [ ]:
# 정류소별

In [12]:
hwasung_station = StationTable[StationTable['시군명']=='화성시']
# 정류소 번호 중복 제거
StationTable.drop_duplicates(subset ="정류소번호", 
                     keep = 'first', inplace = True) 

In [13]:
hwasung_station.shape

(11243, 14)

In [14]:
hwasung_station.sample()

,Unnamed: 0,표준정류장ID,시군명,정류소명,정류소영문명,정류소번호,중앙차로여부,관할관청,위치,WGS84위도,WGS84경도,모바일정류장ID,이비카드정류장ID,station_id
26917,26917,233002321,화성시,비봉교차로,Bibong Junction,55226.0,노변정류장,경기도 화성시,경기도 화성시 비봉면,37.238833,126.8792,[None None None None None None None None None ...,4120152.0,233002321


~~쓸모없는 짓~~

In [15]:
tmp = pd.merge(hwasung_station,RouteStationInfo, how='left', left_on = 'station_id', right_on = 'station_id')

In [16]:
tmp[(tmp['WGS84위도'].isnull())==True].shape

(1776, 20)

In [17]:
tmp.shape

(105131, 20)

In [18]:
tmp2 = tmp.dropna(subset=['WGS84위도'])

In [19]:
tmp2.shape

(103355, 20)

---

In [20]:
TripChain.iloc[0]

암호화카드번호         900079696430
트랜잭션ID                    56
환승횟수                       2
교통카드발행사ID            9000923
총이용객수                      1
사용자구분                      1
교통수단CD1                  500
교통수단CD2                  500
교통수단CD3                  NaN
교통수단CD4                  NaN
교통수단CD5                  NaN
버스노선ID1           4.1002e+07
버스노선ID2           4.1002e+07
버스노선ID3                  NaN
버스노선ID4                  NaN
버스노선ID5                  NaN
차량ID1            1.41772e+08
차량ID2            1.41772e+08
차량ID3                    NaN
차량ID4                    NaN
차량ID5                    NaN
총통행거리                  11170
총탑승시간                     25
총소요시간                     25
승차일시1            2.01807e+13
승차일시2            2.01807e+13
승차일시3                    NaN
승차일시4                    NaN
승차일시5                    NaN
하차일시1            2.01807e+13
하차일시2            2.01807e+13
하차일시3                    NaN
하차일시4                    NaN
하차일시5                    NaN
최초승차일시        

In [22]:
# 시간대별로 특정 노선이 이용되는 ~
# 버스 라인마다의 특성
# 학교 가 들어가는 정류장을 지나가는 버스의 노선 확인 (활성화 여부도 함께 확인)
# 1. 주로 노인이 타는버스 - 저상버스
# 2. 주로 학생이 타는 버스
# 버스 몇인승인지 확인
# 정류소 이름 활용
# 환승설계 - 긴 노선을 짤라서 환승. 미니 메트로

# 운수사명 확인 후 화성 버스만 사용

In [23]:
RouteStationMapping['운수사명'].unique()

array(['경원여객M', '강화운수', '경기고속', '대원고속', '대원버스', '대원운수', '경남여객', '경원여객',
       '경진여객', '금강고속', '김포운수', '남양여객', '대명운수', '대양운수', '도원교통', '명성운수',
       '명진여객', '백성운수', '부천버스', '삼경운수', '삼영운수', '서울고속', '선진시내버스', '성남시내버스',
       '성우운수', '소신여객', '수원여객', '시흥교통', '신성여객', '신일여객', '연천교통', '용남고속',
       '진흥고속', '태화상운', '평안운수', '포천교통', '포천상운', '협진여객', '화영운수', '경기여객',
       '용남버스', '보영운수', '성광운수', '양주교통', '진명여객', '제부여객', '부일교통소사', '부일교통작동',
       '평택여객', '고양교통', '청우운수', '부일교통상동', '오산교통', '화성운수', '선진버스', '경기버스',
       '경기운수', '경기상운', '파주선진', '(주)가온누리엠', '(주)화성여객', '과천여객', '서현운수',
       '코레일네트웍스', '신성교통', '군포여객'], dtype=object)

In [24]:
TripChain['암호화카드번호'].nunique()

498764

# 필요한 데이터만 뽑은 csv 파일 생성 및 사용

```
환승 횟수가 많은 이용객 분석

필요한 것
TripChain > 이용객 정보
            암호화카드번호ID, 버스노선ID 1~5, 총통행거리, 총탑승시간,총소요시간, 승차역1~5, 하차역1~5, 
            최초승차일시, 최종하차일시, 최초승차역ID, 최종하차역ID, 총이용금액, 수집건수, 총이용객수, 사용자구분
            ( 1: 일반, 2: 어린이, 4: 청소년, 8: 경로 )
RouteStationMapping > 이용객이 어떤 버스를 탔는지 확인
                      표준노선ID -> TripChain의 버스노선ID와 merge
                      이비노선ID, 노선명
RouteStationInfo > 이용객이 탄 버스 정류장이 어딘지 저장
```

In [215]:
TripChain_over3 = TripChain[TripChain['환승횟수']>=3]

In [216]:
merge1 = pd.merge(
    TripChain_over3[['암호화카드번호', '버스노선ID1', '버스노선ID2', '버스노선ID3', '버스노선ID4', '버스노선ID5',
                     '총통행거리', '총탑승시간','총소요시간', '승차역ID1', '승차역ID2', '승차역ID3', '승차역ID4',
                     '승차역ID5', '하차역ID1', '하차역ID2', '하차역ID3', '하차역ID4', '하차역ID5',
                     '최초승차일시', '최종하차일시', '최초승차역ID', '최종하차역ID', '총이용금액', '수집건수', 
                     '총이용객수','사용자구분']],
RouteStationMapping[['표준노선ID', '이비노선ID', '노선명']], how = 'left', left_on = '버스노선ID1', 
                        right_on = '표준노선ID')

### 암호화카드번호와 최초승차일시가 같은 데이터는 제외

In [217]:
merge1.shape

(109881, 30)

In [218]:
merge1.drop_duplicates(subset =["암호화카드번호","최초승차일시", "최종하차일시"], 
                     keep = 'first', inplace = True) 
merge1.shape

(98393, 30)

In [219]:
merge1['최초승차역ID'] = merge1['승차역ID1']

In [220]:
merge1 = merge1.dropna(subset=['노선명'])
merge1.head()

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수,총이용객수,사용자구분,표준노선ID,이비노선ID,노선명
0,900079705567,41067109.0,NaN,41021002.0,NaN,NaN,24480,39,39,4197606.0,...,20180701085359,4197606.0,4116627.0,1550,3,1,1,41067109.0,223000032.0,701
2,900079981594,41061025.0,NaN,41425003.0,NaN,NaN,19971,71,71,4108111.0,...,20180701202740,4108111.0,9119276.0,1450,3,1,1,41061025.0,200000105.0,15
3,900030101425,41027009.0,41031009.0,NaN,NaN,NaN,12560,45,45,4100114.0,...,20180701140028,4100114.0,1713.0,1350,3,1,1,41027009.0,200000028.0,301
5,900030101881,41008148.0,NaN,28222001.0,NaN,NaN,64294,130,130,4131032.0,...,20180701210709,4131032.0,2806914.0,2350,3,1,1,41008148.0,233000083.0,30
7,900030102978,41025004.0,41003010.0,41031019.0,41036110.0,NaN,62600,165,165,4176873.0,...,20180701182312,4176873.0,4116855.0,3100,4,1,1,41025004.0,227000006.0,30-3


In [221]:
merge1.shape

(69927, 30)

In [222]:
del merge1['표준노선ID']

In [223]:
del merge1['이비노선ID']

In [224]:
merge1['버스노선ID1'] = merge1['노선명']

In [225]:
del merge1['노선명']

In [226]:
merge2 = pd.merge(merge1[['암호화카드번호', '버스노선ID1', '버스노선ID2', '버스노선ID3', '버스노선ID4', '버스노선ID5',
          '총통행거리', '총탑승시간','총소요시간', '승차역ID1', '승차역ID2', '승차역ID3', '승차역ID4',
           '승차역ID5', '하차역ID1', '하차역ID2', '하차역ID3', '하차역ID4', '하차역ID5',
           '최초승차일시', '최종하차일시', '최초승차역ID', '최종하차역ID', '총이용금액', '수집건수',
                         '총이용객수','사용자구분']],
RouteStationMapping[['표준노선ID', '이비노선ID', '노선명']], how = 'left', left_on = '버스노선ID2', 
                        right_on = '표준노선ID')

In [227]:
merge2.head()

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수,총이용객수,사용자구분,표준노선ID,이비노선ID,노선명
0,900079705567,701,NaN,41021002.0,NaN,NaN,24480,39,39,4197606.0,...,20180701085359,4197606.0,4116627.0,1550,3,1,1,NaN,NaN,NaN
1,900079981594,15,NaN,41425003.0,NaN,NaN,19971,71,71,4108111.0,...,20180701202740,4108111.0,9119276.0,1450,3,1,1,NaN,NaN,NaN
2,900030101425,301,41031009.0,NaN,NaN,NaN,12560,45,45,4100114.0,...,20180701140028,4100114.0,1713.0,1350,3,1,1,41031009.0,200000049.0,112
3,900030101881,30,NaN,28222001.0,NaN,NaN,64294,130,130,4131032.0,...,20180701210709,4131032.0,2806914.0,2350,3,1,1,NaN,NaN,NaN
4,900030102978,30-3,41003010.0,41031019.0,41036110.0,NaN,62600,165,165,4176873.0,...,20180701182312,4176873.0,4116855.0,3100,4,1,1,41003010.0,234000310.0,1009


In [228]:
merge2['버스노선ID2'] = merge2['노선명']
merge2 = merge2.dropna(subset=['노선명'])
del merge2['노선명']
del merge2['표준노선ID']
del merge2['이비노선ID']
merge2.head()

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,하차역ID4,하차역ID5,최초승차일시,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수,총이용객수,사용자구분
2,900030101425,301,112,NaN,NaN,NaN,12560,45,45,4100114.0,...,NaN,NaN,20180701124244,20180701140028,4100114.0,1713.0,1350,3,1,1
4,900030102978,30-3,1009,41031019.0,41036110.0,NaN,62600,165,165,4176873.0,...,4116855.0,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4,1,1
6,900480680100,13-1,M5107,11110052.0,11110055.0,NaN,45645,66,66,4108218.0,...,9034961.0,NaN,20180701144127,20180701155848,4108218.0,9034961.0,2800,4,1,1
10,900508818160,7,202,41031107.0,NaN,NaN,10930,24,24,4108196.0,...,NaN,NaN,20180701204621,20180701213623,4108196.0,4199077.0,1350,3,1,1
11,900351733805,13-5,400-4,41036128.0,NaN,NaN,16150,49,49,4108106.0,...,NaN,NaN,20180701145032,20180701161318,4108106.0,4102942.0,1450,3,1,1


In [229]:
merge2.shape

(43741, 27)

In [230]:
merge3 = pd.merge(merge2[['암호화카드번호', '버스노선ID1', '버스노선ID2', '버스노선ID3', '버스노선ID4', '버스노선ID5',
          '총통행거리', '총탑승시간','총소요시간', '승차역ID1', '승차역ID2', '승차역ID3', '승차역ID4',
           '승차역ID5', '하차역ID1', '하차역ID2', '하차역ID3', '하차역ID4', '하차역ID5',
           '최초승차일시', '최종하차일시', '최초승차역ID', '최종하차역ID', '총이용금액', '수집건수',
                         '총이용객수','사용자구분']],
RouteStationMapping[['표준노선ID', '이비노선ID', '노선명']], how = 'left', left_on = '버스노선ID3', 
                        right_on = '표준노선ID')

In [231]:
merge3.head()

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수,총이용객수,사용자구분,표준노선ID,이비노선ID,노선명
0,900030101425,301,112,NaN,NaN,NaN,12560,45,45,4100114.0,...,20180701140028,4100114.0,1713.0,1350,3,1,1,NaN,NaN,NaN
1,900030102978,30-3,1009,41031019.0,41036110.0,NaN,62600,165,165,4176873.0,...,20180701182312,4176873.0,4116855.0,3100,4,1,1,41031019.0,200000025.0,92
2,900030102978,30-3,1009,41031019.0,41036110.0,NaN,62600,165,165,4176873.0,...,20180701182312,4176873.0,4116855.0,3100,4,1,1,41031019.0,200000025.0,92
3,900480680100,13-1,M5107,11110052.0,11110055.0,NaN,45645,66,66,4108218.0,...,20180701155848,4108218.0,9034961.0,2800,4,1,1,NaN,NaN,NaN
4,900508818160,7,202,41031107.0,NaN,NaN,10930,24,24,4108196.0,...,20180701213623,4108196.0,4199077.0,1350,3,1,1,41031107.0,200000077.0,5-1


In [232]:
merge3 = merge3.dropna(subset=['노선명'])
merge3['버스노선ID3'] = merge3['노선명']
del merge3['노선명']
del merge3['표준노선ID']
del merge3['이비노선ID']

In [233]:
merge3.shape

(34602, 27)

### 환승횟수가 4인 데이터

In [234]:
TripChain_4 = merge3[merge3['수집건수']==4]

In [235]:
TripChain_4.head()

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,하차역ID4,하차역ID5,최초승차일시,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수,총이용객수,사용자구분
1,900030102978,30-3,1009,92,41036110.0,NaN,62600,165,165,4176873.0,...,4116855.0,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4,1,1
2,900030102978,30-3,1009,92,41036110.0,NaN,62600,165,165,4176873.0,...,4116855.0,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4,1,1
32,900480674905,51,720-1,700-2,41419002.0,NaN,14050,42,42,4110270.0,...,4115063.0,NaN,20180701074728,20180701091223,4110270.0,4115063.0,1350,4,1,1
37,900469148309,98,777,310,41006241.0,NaN,12030,41,41,4116947.0,...,NaN,NaN,20180701144949,20180701160447,4116947.0,NaN,1350,4,1,1
46,900526791421,62,707,11,41036111.0,NaN,28710,78,78,4105239.0,...,4100050.0,NaN,20180701172810,20180701185700,4105239.0,4100050.0,2050,4,1,1


In [236]:
merge4 = pd.merge(TripChain_4[['암호화카드번호', '버스노선ID1', '버스노선ID2', '버스노선ID3', '버스노선ID4', '버스노선ID5',
          '총통행거리', '총탑승시간','총소요시간', '승차역ID1', '승차역ID2', '승차역ID3', '승차역ID4',
           '승차역ID5', '하차역ID1', '하차역ID2', '하차역ID3', '하차역ID4', '하차역ID5',
           '최초승차일시', '최종하차일시', '최초승차역ID', '최종하차역ID', '총이용금액', '수집건수',
                              '총이용객수','사용자구분']],
RouteStationMapping[['표준노선ID', '이비노선ID', '노선명']], how = 'left', left_on = '버스노선ID4', 
                        right_on = '표준노선ID')
merge4.head()

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수,총이용객수,사용자구분,표준노선ID,이비노선ID,노선명
0,900030102978,30-3,1009,92,41036110.0,NaN,62600,165,165,4176873.0,...,20180701182312,4176873.0,4116855.0,3100,4,1,1,41036110.0,200000097.0,2-1
1,900030102978,30-3,1009,92,41036110.0,NaN,62600,165,165,4176873.0,...,20180701182312,4176873.0,4116855.0,3100,4,1,1,41036110.0,200000097.0,2-1
2,900030102978,30-3,1009,92,41036110.0,NaN,62600,165,165,4176873.0,...,20180701182312,4176873.0,4116855.0,3100,4,1,1,41036110.0,200000097.0,2-1
3,900030102978,30-3,1009,92,41036110.0,NaN,62600,165,165,4176873.0,...,20180701182312,4176873.0,4116855.0,3100,4,1,1,41036110.0,200000097.0,2-1
4,900480674905,51,720-1,700-2,41419002.0,NaN,14050,42,42,4110270.0,...,20180701091223,4110270.0,4115063.0,1350,4,1,1,NaN,NaN,NaN


In [237]:
merge4['버스노선ID4'] = merge4['노선명']
merge4 = merge4.dropna(subset=['노선명'])
del merge4['노선명']
del merge4['표준노선ID']
del merge4['이비노선ID']

In [238]:
merge4.head()

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,하차역ID4,하차역ID5,최초승차일시,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수,총이용객수,사용자구분
0,900030102978,30-3,1009,92,2-1,NaN,62600,165,165,4176873.0,...,4116855.0,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4,1,1
1,900030102978,30-3,1009,92,2-1,NaN,62600,165,165,4176873.0,...,4116855.0,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4,1,1
2,900030102978,30-3,1009,92,2-1,NaN,62600,165,165,4176873.0,...,4116855.0,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4,1,1
3,900030102978,30-3,1009,92,2-1,NaN,62600,165,165,4176873.0,...,4116855.0,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4,1,1
5,900469148309,98,777,310,66,NaN,12030,41,41,4116947.0,...,NaN,NaN,20180701144949,20180701160447,4116947.0,NaN,1350,4,1,1


In [239]:
merge4.shape

(5122, 27)

### 환승횟수가 5인 데이터

In [240]:
TripChain_5 = merge3[merge3['수집건수']==5]

In [241]:
merge5_1 = pd.merge(TripChain_5[['암호화카드번호', '버스노선ID1', '버스노선ID2', '버스노선ID3', '버스노선ID4', '버스노선ID5',
          '총통행거리', '총탑승시간','총소요시간', '승차역ID1', '승차역ID2', '승차역ID3', '승차역ID4',
           '승차역ID5', '하차역ID1', '하차역ID2', '하차역ID3', '하차역ID4', '하차역ID5',
           '최초승차일시', '최종하차일시', '최초승차역ID', '최종하차역ID', '총이용금액', '수집건수',
                                '총이용객수','사용자구분']],
RouteStationMapping[['표준노선ID', '이비노선ID', '노선명']], how = 'left', left_on = '버스노선ID4', 
                        right_on = '표준노선ID')
merge5_1.head()

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수,총이용객수,사용자구분,표준노선ID,이비노선ID,노선명
0,900097650068,9-3,7770,64,41031030.0,41008007.0,54250,88,88,4103271.0,...,20180701112742,4103271.0,4116659.0,2900,5,1,1,41031030.0,200000055.0,25
1,900029059863,4-1,2-1,98,41031040.0,41006241.0,15080,58,58,4170101.0,...,20180701161148,4170101.0,4116887.0,1030,5,1,4,41031040.0,200000078.0,62-1
2,900029059863,4-1,2-1,98,41031040.0,41006241.0,15080,58,58,4170101.0,...,20180701161148,4170101.0,4116887.0,1030,5,1,4,41031040.0,200000078.0,62-1
3,900029059863,4-1,2-1,98,41031040.0,41006241.0,15080,58,58,4170101.0,...,20180701161148,4170101.0,4116887.0,1030,5,1,4,41031040.0,200000078.0,62-1
4,900029059863,4-1,2-1,98,41031040.0,41006241.0,15080,58,58,4170101.0,...,20180701161148,4170101.0,4116887.0,1030,5,1,4,41031040.0,200000078.0,62-1


In [242]:
merge5_1['버스노선ID4'] = merge5_1['노선명']
merge5_1 = merge5_1.dropna(subset=['노선명'])
del merge5_1['노선명']
del merge5_1['표준노선ID']
del merge5_1['이비노선ID']
merge5_1.shape

(1491, 27)

In [243]:
merge5_2 = pd.merge(merge5_1[['암호화카드번호', '버스노선ID1', '버스노선ID2', '버스노선ID3', '버스노선ID4', '버스노선ID5',
          '총통행거리', '총탑승시간','총소요시간', '승차역ID1', '승차역ID2', '승차역ID3', '승차역ID4',
           '승차역ID5', '하차역ID1', '하차역ID2', '하차역ID3', '하차역ID4', '하차역ID5',
           '최초승차일시', '최종하차일시', '최초승차역ID', '최종하차역ID', '총이용금액', '수집건수',
                             '총이용객수','사용자구분']],
RouteStationMapping[['표준노선ID', '이비노선ID', '노선명']], how = 'left', left_on = '버스노선ID5', 
                        right_on = '표준노선ID')
merge5_2.head()

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수,총이용객수,사용자구분,표준노선ID,이비노선ID,노선명
0,900097650068,9-3,7770,64,25,41008007.0,54250,88,88,4103271.0,...,20180701112742,4103271.0,4116659.0,2900,5,1,1,41008007.0,233000031.0,7770
1,900029059863,4-1,2-1,98,62-1,41006241.0,15080,58,58,4170101.0,...,20180701161148,4170101.0,4116887.0,1030,5,1,4,41006241.0,228000001.0,66
2,900029059863,4-1,2-1,98,62-1,41006241.0,15080,58,58,4170101.0,...,20180701161148,4170101.0,4116887.0,1030,5,1,4,41006241.0,228000001.0,66
3,900029059863,4-1,2-1,98,62-1,41006241.0,15080,58,58,4170101.0,...,20180701161148,4170101.0,4116887.0,1030,5,1,4,41006241.0,228000001.0,66
4,900029059863,4-1,2-1,98,62-1,41006241.0,15080,58,58,4170101.0,...,20180701161148,4170101.0,4116887.0,1030,5,1,4,41006241.0,228000001.0,66


In [244]:
merge5_2['버스노선ID5'] = merge5_2['노선명']
merge5_2 = merge5_2.dropna(subset=['노선명'])
del merge5_2['노선명']
del merge5_2['표준노선ID']
del merge5_2['이비노선ID']
merge5_2.shape

(1490, 27)

In [245]:
merge5_2.head()

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,하차역ID4,하차역ID5,최초승차일시,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수,총이용객수,사용자구분
0,900097650068,9-3,7770,64,25,7770,54250,88,88,4103271.0,...,4100126.0,4116659.0,20180701092638,20180701112742,4103271.0,4116659.0,2900,5,1,1
1,900029059863,4-1,2-1,98,62-1,66,15080,58,58,4170101.0,...,4170102.0,4116887.0,20180701141312,20180701161148,4170101.0,4116887.0,1030,5,1,4
2,900029059863,4-1,2-1,98,62-1,66,15080,58,58,4170101.0,...,4170102.0,4116887.0,20180701141312,20180701161148,4170101.0,4116887.0,1030,5,1,4
3,900029059863,4-1,2-1,98,62-1,66,15080,58,58,4170101.0,...,4170102.0,4116887.0,20180701141312,20180701161148,4170101.0,4116887.0,1030,5,1,4
4,900029059863,4-1,2-1,98,62-1,66,15080,58,58,4170101.0,...,4170102.0,4116887.0,20180701141312,20180701161148,4170101.0,4116887.0,1030,5,1,4


### 환승횟수 3개 이상인 데이터들 다같이 concat

In [246]:
tmp_merge3 = merge3[merge3['수집건수']==3]

In [247]:
del all_merge
# all_merge.head()

In [248]:
all_merge = pd.concat([tmp_merge3, merge4])

In [249]:
all_merge = pd.concat([all_merge, merge5_2])

In [250]:
all_merge.shape

(34122, 27)

In [251]:
all_merge.drop_duplicates(subset =["암호화카드번호","최초승차일시", "최종하차일시"], 
                     keep = 'first', inplace = True) 

In [252]:
all_merge.shape

(24745, 27)

In [253]:
all_merge

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,하차역ID4,하차역ID5,최초승차일시,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수,총이용객수,사용자구분
4,900508818160,7,202,5-1,NaN,NaN,10930,24,24,4108196.0,...,NaN,NaN,20180701204621,20180701213623,4108196.0,4199077.0,1350,3,1,1
5,900351733805,13-5,400-4,22,NaN,NaN,16150,49,49,4108106.0,...,NaN,NaN,20180701145032,20180701161318,4108106.0,4102942.0,1450,3,1,1
6,900351733805,11,13-5,13-4,NaN,NaN,11350,24,24,4105311.0,...,NaN,NaN,20180701212521,20180701221059,4105311.0,4117053.0,1350,3,1,1
8,900028085150,62-1,88-1,11-1,NaN,NaN,7140,26,26,4116853.0,...,NaN,NaN,20180701182945,20180701191500,4116853.0,4170093.0,870,3,1,4
9,900028088225,720-2,32,30-1,NaN,NaN,13690,46,46,4170102.0,...,NaN,NaN,20180701141014,20180701150226,4170102.0,4130061.0,1350,3,1,1
11,900497300024,11,720-2,3,NaN,NaN,11070,35,35,4105379.0,...,NaN,NaN,20180701080857,20180701085254,4105379.0,4116875.0,1350,3,1,1
13,900029953051,98,5,301,NaN,NaN,6930,25,25,4116719.0,...,NaN,NaN,20180701192632,20180701195748,4116719.0,4108093.0,1250,3,1,1
16,900029249197,116-1,116-3,50,NaN,NaN,28670,102,102,4176773.0,...,NaN,NaN,20180701110122,20180701125354,4176773.0,4123044.0,1650,3,1,1
18,900469599427,16-2,92-1,13-1,NaN,NaN,10570,29,29,4102825.0,...,NaN,NaN,20180701063925,20180701071516,4102825.0,4160314.0,1350,3,1,1
21,900030389260,310,83-1,27,NaN,NaN,13860,45,45,4122214.0,...,NaN,NaN,20180701215057,20180701231539,4122214.0,4150199.0,1350,3,1,1


In [260]:
# all_merge[all_merge['사용자구분']==1]
all_merge[all_merge['총이용객수']!=1]
# 혼자 탄게 아닌 분들은...? 흠

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,하차역ID4,하차역ID5,최초승차일시,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수,총이용객수,사용자구분
143,900473376586,5,900,99-2,NaN,NaN,14450,41,41,4101430.0,...,NaN,NaN,20180701180003,20180701184311,4101430.0,4116729.0,2700,3,2,1
150,900168187974,34,92-1,73-1,NaN,NaN,9130,35,35,4116875.0,...,NaN,NaN,20180701165818,20180701174114,4116875.0,4130114.0,2500,3,2,1
243,900483351296,13,37,62-1,NaN,NaN,3410,13,13,4107936.0,...,NaN,NaN,20180701161948,20180701165932,4107936.0,4112024.0,1740,3,2,4
372,900511532142,38,301,2007,NaN,NaN,48870,86,86,4130354.0,...,NaN,NaN,20180701130546,20180701145123,4130354.0,4100164.0,5600,3,2,1
584,900497491048,720-1,22,65,NaN,NaN,2410,8,8,4115594.0,...,NaN,NaN,20180701185158,20180701192356,4115594.0,4116922.0,2500,3,2,1
1001,900523724831,720-2,7-2,7-1,NaN,NaN,5730,18,18,4117072.0,...,NaN,NaN,20180701141845,20180701145529,4117072.0,4170094.0,2500,3,2,1
1003,900461791875,3,700-2,2-1,NaN,NaN,5360,20,20,4116787.0,...,NaN,NaN,20180701185437,20180701195551,4116787.0,4116789.0,1740,3,2,4
1075,900466648832,99-2,720-2,3,NaN,NaN,3310,11,11,4116806.0,...,NaN,NaN,20180701192510,20180701195344,4116806.0,4116894.0,2120,3,2,1
1487,900395833455,99,80,18,NaN,NaN,9550,35,35,4160224.0,...,NaN,NaN,20180701174219,20180701182843,4160224.0,4197689.0,2500,3,2,1
1590,100033235000,82-1,88,98,NaN,NaN,8250,31,31,4108064.0,...,NaN,NaN,20180701183204,20180701190924,4108064.0,4108158.0,2500,3,2,1


In [264]:
all_merge[all_merge['사용자구분']==4].shape

(1037, 27)

In [259]:
# all_merge.to_csv('Over3Transfer.csv', sep=',')

```
환승 횟수가 3 이상이고 하차태그를 다 찍은 사람은 총 24745명
1일반 23678
2어린이 30
4청소년 1037
8경로 0명


이 사람들이 보통 언제타는지, 어떤 버스를 타는지, 얼마나 오래 타는지
```

### 정류소 연결... ㅎㄷㄷ

In [267]:
RouteStationInfo.sample()

,seq,pr_station_id,bus_line_no,bus_line_no_seq,station_nm,station_id,mobile_no
12181,95649,234000028,10-5,84,수락산등산로입구,222000595,23005.0


In [269]:
StationTable.sample()

,Unnamed: 0,표준정류장ID,시군명,정류소명,정류소영문명,정류소번호,중앙차로여부,관할관청,위치,WGS84위도,WGS84경도,모바일정류장ID,이비카드정류장ID,station_id
10996,10996,231000672,안성시,방신리,Bangsin-ri,34062.0,노변정류장,경기도 안성시,경기도 안성시 양성면,37.036,127.18355,34062,NaN,231000672


In [270]:
hwasung_station.sample()

,Unnamed: 0,표준정류장ID,시군명,정류소명,정류소영문명,정류소번호,중앙차로여부,관할관청,위치,WGS84위도,WGS84경도,모바일정류장ID,이비카드정류장ID,station_id
34242,34242,233001387,화성시,어은삼거리,Eoeun Samgeori,37593.0,노변정류장,경기도 화성시,경기도 화성시 장안면,37.0862,126.826533,[None None None None None None None None None ...,4171014.0,233001387


```
필요한 데이터
hwasung_station > 
이비카드 정류장 ID (승하차 정류장과 연결), 표준정류장ID (1, 2, 3, 4, 5로 만들자..?), 정류소명, 위도, 경도
```

# 정류소명 이용